# Depth video processing

Copyright (c) 2017 Alexander Menkin
Use of this source code is governed by an MIT-style license that can be found in the LICENSE file at
https://github.com/miloiloloo/diploma_2017_method/blob/master/LICENSE

In [ ]:
import numpy as np
import ffms
from skimage import measure, filters, morphology
from scipy.optimize import linear_sum_assignment
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [ ]:
def get_z_image(depth_frame_from_kinect):
    ''' Handle depth frame from kinect '''
    image16bit = np.ndarray(shape=(frame_height, frame_width), dtype='<u2', buffer=depth_frame_from_kinect.planes[0])
    z_image = np.zeros(shape=image16bit.shape, dtype=np.float32)
    z_image = image16bit * float(0.001)
    indecies = (z_image != 0)
    z_image[indecies] = z_image[indecies] - float(0.024)
    ''' OUTPUT '''
    return z_image

In [ ]:
def get_mass_center(foreground, previous_center, max_center_movement):
    ''' To find a mouse's region in image and count its center of mass
        It rerurns (has_been_found, mass_center)    '''
    
    ''' Predefined level for threshold '''
    threshold_level = 0.025
    ''' Predefined min area (number of pixels) of region '''
    min_region_area = 20
    
    ''' Threshold '''
    threshold_image = np.zeros(shape=foreground.shape, dtype=np.uint16)
    threshold_image[foreground > threshold_level] = 1
    label_image = measure.label(threshold_image)
    regions = []
    ''' Check all regions '''
    for region in measure.regionprops(label_image):
        if region.area >= min_region_area:
            ''' Region is enough big '''
            regions = regions + [region]
    region_lengths = np.ndarray(shape=(len(regions)), dtype=np.int16)
    for region_index in range(0, len(regions)):
        region_lengths[region_index] = regions[region_index].area
    ''' Get indecies of regions in order of decrease their areas '''
    arglist = np.argsort(region_lengths)[::-1]
    
    ''' From the biggest region to the smallest '''
    for region_index in arglist:
        region = regions[region_index]
        ''' Count E(x,y) '''
        center = np.zeros(shape=(2), dtype=np.float32)
        sum_of_z_values = 0
        for coord in region.coords:
            z_value = foreground[coord[0], coord[1]]
            sum_of_z_values = sum_of_z_values + z_value
            center = center + z_value * coord
        center = center / sum_of_z_values
        ''' Check the distance between its center and previous center '''
        if np.linalg.norm(previous_center - center) < max_center_movement:
            ''' Acceptable center has been found '''
            return (True, center)
    
    ''' Acceptable center has not been found '''
    return (False, np.zeros(shape=(2), dtype=np.int16))

In [ ]:
def get_mass_centers(foreground):
    ''' Get centers of mass for enough good regions
        It returns array of mass centers (from the biggest region to the smallest (but with area more then min_region_area))
    '''
    
    ''' Predefined level for threshold '''
    threshold_level = 0.025
    ''' Predefined min area (number of pixels) of region '''
    min_region_area = 20
    
    ''' Threshold '''
    threshold_image = np.zeros(shape=foreground.shape, dtype=np.uint16)
    threshold_image[foreground > threshold_level] = 1
    label_image = measure.label(threshold_image)
    regions = []
    ''' Check all regions '''
    for region in measure.regionprops(label_image):
        if region.area >= min_region_area:
            ''' Region is enough big '''
            regions = regions + [region]
    region_lengths = np.ndarray(shape=(len(regions)), dtype=np.int16)
    for region_index in range(0, len(regions)):
        region_lengths[region_index] = regions[region_index].area
    ''' Get indecies of regions in order of decrease their areas '''
    arglist = np.argsort(region_lengths)[::-1]
    centers = []
    
    ''' From the biggest region to the smallest '''
    for region_index in arglist:
        region = regions[region_index]
        ''' Count E(x,y) '''
        center = np.zeros(shape=(2), dtype=np.float32)
        sum_of_z_values = 0
        for coord in region.coords:
            z_value = foreground[coord[0], coord[1]]
            sum_of_z_values = sum_of_z_values + z_value
            center = center + z_value * coord
        center = center / sum_of_z_values
        centers = centers + [center]    
    
    ''' OUTPUT '''
    return centers

In [ ]:
def get_foreground(z_image, background):
    ''' Background substruction for kinect z_image
        It returns foreground
    '''
    
    ''' CHECK INPUT '''   
    ''' Incorrect input 1: z_image's (height, width) is not equal background's (height, width) '''
    if z_image.shape != background.shape:
        raise Exception
    
    ''' ALGORITHM & OUTPUT '''
    ''' To ignore zeros as incorrect data '''
    indecies = z_image == 0
    z_image[indecies] = background[indecies]
    
    ''' Substruction: background is further then foreground, that's why substruction is done in such a way '''
    return background - z_image

In [ ]:
def get_assemblies(depth_source,
                   frame_width,
                   frame_height,
                   first_frame_idx,
                   last_frame_idx,
                   neighbour_offset,
                   number_of_left_neighbours,
                   number_of_right_neighbours,
                   patch_width,
                   patch_height,
                   background,
                   skipped_frames_limit,
                   previous_center,
                   max_center_movement):
    ''' Get assemblies from depth_source
        It returns (assemblies, indecies, skipped_indecies)
    '''
    
    ''' CHECK INPUT '''
    ''' Incorrect input 1: incorrect frame size '''
    if frame_width <= 0 or frame_height <= 0:
        raise Exception
    ''' Incorrect input 2: index can't be less then 0 '''
    if first_frame_idx < 0:
        raise Exception
    ''' Incorrect input 3: index can't be more then depth_source.properties.NumFrames '''
    if last_frame_idx >= depth_source.properties.NumFrames:
        raise Exception
    ''' Incorrect input 4: first index must be less then last index '''
    if first_frame_idx > last_frame_idx:
        raise Exception
    ''' Incorrect input 5: neighbour_offset can't be less then 0 or equal 0'''
    if neighbour_offset <= 0:
        raise Exception
    ''' Incorrect input 6: number of neighbours can't be less then 0 '''
    if number_of_left_neighbours < 0 or number_of_right_neighbours < 0:
        raise Exception
    ''' Incorrect input 7: incorrect patch size: it must be more then 0 and less then frame_size '''
    if patch_width <= 0 or patch_height <= 0 or patch_width > frame_width or patch_height > frame_height:
        raise Exception
    ''' Incorrect input 8: background must have the same size with frame '''
    if background.shape[0] != frame_height or background.shape[1] != frame_width:
        raise Exception
    ''' Incorrect input 9: limit can't be less then 0 '''
    if skipped_frames_limit < 0:
        raise Exception
    ''' Incorrect input 10: previous_center must be inside of frame '''
    if previous_center[0] < 0 or previous_center[0] >= frame_height or previous_center[1] < 0 or previous_center[1] >= frame_width:
        raise Exception
    ''' Incorrect input 11: movement is a module of vector and can't be less then 0 '''
    if max_center_movement < 0:
        raise Exception
        
    ''' ALGORITHM '''
    ''' Window size '''
    window_size = number_of_left_neighbours*neighbour_offset + number_of_right_neighbours*neighbour_offset + 1
    
    ''' Max number of assemblies: if max number of assemblies is not more then 0, function returns empty arrays '''
    max_number_of_assemblies = (last_frame_idx - first_frame_idx + 1) - window_size + 1
    if max_number_of_assemblies <= 0:
        ''' EMPTY OUTPUT CONDITION '''
        return (np.array([], dtype=np.float32), np.array([], dtype=np.int16), np.array([], dtype=np.int16))
    
    ''' Number of patches in assembly '''
    number_of_patches_in_assembly = number_of_left_neighbours + number_of_right_neighbours + 1
    
    ''' Indecies and assemblies '''
    indecies = np.ndarray(shape=(max_number_of_assemblies), dtype=np.int16)
    assemblies = np.ndarray(shape=(max_number_of_assemblies, patch_width, patch_height, number_of_patches_in_assembly), dtype=np.float32)
    number_of_assemblies = 0
    
    ''' Window '''
    window = np.ndarray(shape=(frame_height, frame_width, window_size), dtype=np.float32)
    next_frame_to_write_in_window_idx = first_frame_idx
    ''' Init window '''
    for window_idx in range(0, window_size):
        ''' Get #next_frame_to_write_in_window_idx frame from depth_source '''
        frame = depth_source.get_frame(next_frame_to_write_in_window_idx)
        ''' Check frame size '''
        if frame.EncodedWidth != frame_width or frame.EncodedHeight != frame_height:
            ''' Incorrect depth_source: Unexpected frame size '''
            raise Exception
        ''' Correct frame size '''
        window[:, :, window_idx] = get_foreground(get_z_image(frame), background)
        next_frame_to_write_in_window_idx = next_frame_to_write_in_window_idx + 1
    
    ''' Offsets of patch's borders relative to the patch's center '''
    ''' Min x patch's border's offset '''
    min_x_patch_border_offset = 0
    if patch_width % 2 == 0:
        min_x_patch_border_offset = -(patch_width/2 - 1)
    else:
        min_x_patch_border_offset = -((patch_width - 1)/2)
    ''' Min y patch's border's offset '''
    min_y_patch_border_offset = 0
    if patch_height % 2 == 0:
        min_y_patch_border_offset = -(patch_height/2 - 1)
    else:
        min_y_patch_border_offset = -((patch_height - 1)/2)
    ''' Max x and y patch's border's offsets '''
    max_x_patch_border_offset = min_x_patch_border_offset + patch_width - 1
    max_y_patch_border_offset = min_y_patch_border_offset + patch_height - 1
    
    ''' Window parameters: to work with window '''
    window_to_write_next_frame_idx = 0
    window_assembly_indecies = np.arange(0, window_size, neighbour_offset)
    window_current_idx = number_of_left_neighbours*neighbour_offset
    print("assembly_indecies: " + str(window_assembly_indecies))
    print("current_idx: " + str(window_current_idx))
    
    ''' Counter for skipped in a row frames and array of skipped frame's indecies '''
    number_of_skipped_frames_in_a_row = 0
    skipped_indecies = np.zeros(shape=(0), dtype=np.int16)
    
    ''' Current frame index '''
    current_frame_idx = first_frame_idx + number_of_left_neighbours*neighbour_offset
    
    ''' Cycle '''
    number_of_processed_frames = 0
    
    while True:   
        
        ''' Find center '''
        has_been_found, center = get_mass_center(window[:, :, window_current_idx], previous_center, max_center_movement)
        
        ''' Handle result of center finding '''
        if has_been_found:
            ''' Center has been found '''
            ''' Update previous center '''
            previous_center = center
            number_of_skipped_frames_in_a_row = 0
            ''' Count borders of patches '''
            min_x_patch_border = int(round(center[1])) + min_x_patch_border_offset
            max_x_patch_border = int(round(center[1])) + max_x_patch_border_offset
            min_y_patch_border = int(round(center[0])) + min_y_patch_border_offset
            max_y_patch_border = int(round(center[0])) + max_y_patch_border_offset
            ''' If patches are inside of frames, assembly will be created '''
            if min_x_patch_border >= 0 and max_x_patch_border < frame_width and min_y_patch_border >= 0 and max_y_patch_border < frame_height:
                patches = window[min_y_patch_border : max_y_patch_border + 1, min_x_patch_border : max_x_patch_border + 1, window_assembly_indecies]
                assemblies[number_of_assemblies, :, :, :] = np.moveaxis(patches, [0, 1], [1, 0])
                indecies[number_of_assemblies] = current_frame_idx
                number_of_assemblies = number_of_assemblies + 1       
        else:
            ''' Center has not been found '''
            ''' Add 1 to the number of skipped in a row frames and check the limit '''
            skipped_indecies = np.append(skipped_indecies, current_frame_idx)
            number_of_skipped_frames_in_a_row = number_of_skipped_frames_in_a_row + 1
            if number_of_skipped_frames_in_a_row > skipped_frames_limit:
                break
        
        ''' Update number of processed frames and check is it the end '''
        number_of_processed_frames = number_of_processed_frames + 1
        if number_of_processed_frames == max_number_of_assemblies:
            ''' Window is in the end of the frame's sequence '''
            break
        
        ''' Load new image in the window '''
        frame = depth_source.get_frame(next_frame_to_write_in_window_idx)
        
        ''' Check frame size '''
        if frame.EncodedWidth != frame_width or frame.EncodedHeight != frame_height:
            ''' Incorrect depth_source: Unexpected frame size '''
            raise Exception
        ''' Correct frame size '''
        window[:, :, window_to_write_next_frame_idx] = get_foreground(get_z_image(frame), background)
        
        ''' Update window indecies '''
        window_to_write_next_frame_idx = (window_to_write_next_frame_idx + 1) % window_size
        window_assembly_indecies = (window_assembly_indecies + 1) % window_size
        window_current_idx = (window_current_idx + 1) % window_size
        
        ''' Update frame indecies '''
        next_frame_to_write_in_window_idx = next_frame_to_write_in_window_idx + 1
        current_frame_idx = current_frame_idx + 1
        
        ''' End of cycle '''
    
    ''' OUTPUT '''
    return (assemblies[0 : number_of_assemblies, :, :, :],
            indecies[0 : number_of_assemblies],
            skipped_indecies)

In [ ]:
def get_background(depth_source, frame_width, frame_height, frame_indecies):
    ''' Get background using frames from depth_source
        It returns a background
    '''
    
    ''' CHECK INPUT '''
    ''' Incorrect input 1: if frame's indecies length is zero '''
    if frame_indecies.shape[0] == 0:
        raise Exception
    ''' Incorrect input 2: if frame's indecies contain index which is not correct for this depth source '''
    if np.min(frame_indecies) < 0 or np.max(frame_indecies) >= depth_source.properties.NumFrames:
        raise Exception
    
    ''' ALGORITHM '''
    ''' Background set '''
    background_set_size = len(frame_indecies);
    background_set = np.zeros(shape=(background_set_size, frame_height, frame_width), dtype="float32")
    ''' Init background set '''
    for background_set_idx in range(0, background_set_size):
        frame = depth_source.get_frame(frame_indecies[background_set_idx])
        ''' If unexpected frame's width or height '''
        if frame.EncodedWidth != frame_width or frame.EncodedHeight != frame_height:
            raise Exception
        ''' Alright '''
        background_set[background_set_idx, :, :] = get_z_image(frame)
    
    ''' Background '''
    background = np.zeros(shape=(frame_height, frame_width),dtype='float32')

    ''' Median Filter'''
    for i in range(0, frame_height):
        for j in range(0, frame_width):
            pixels = background_set[:, i, j]
            pixels = pixels[pixels > 0]
            if (pixels.shape[0] > 0):
                background[i, j] = np.median(pixels)
    
    ''' OUTPUT '''
    return background

In [ ]:
def write_processed_depth_data(filepath,
                     number_of_frames,
                     assembly_offsets,
                     assemblies,
                     indecies):
    ''' Write assemblies '''
    try:
        f = open(filepath, 'wb')
        save = {
            'number_of_frames': number_of_frames,
            'assembly_offsets': assembly_offsets,
            'assemblies': assemblies,
            'indecies': indecies
        }
        pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
        f.close()
    except Exception as e:
        print('Unable to save data to', filepath, ':', e)
        raise
    print("\nData has been saved")
    print("-------")
    print("file path: " + filepath)
    print("number of frames: " + str(number_of_frames))
    print("assembly offsets: " + str(assembly_offsets))
    print("number of assemblies: " + str(assemblies.shape[0]))
    print("size: " + str(assemblies.shape[1]) + "x" + str(assemblies.shape[2]))
    print("-------\n")
    return

In [ ]:
''' input files paths '''
# Warning: write your own filepaths
input_file_paths = []

''' Containers for video data '''
depth_sources = []
frames_numbers = []
frame_widths = []
frame_heights = []

''' Open videos '''
for input_file_path in input_file_paths:
    ''' Index '''
    indexer = ffms.Indexer(input_file_path)
    index = indexer.do_indexing(-1)
    ''' Depth source '''
    depth_source = ffms.VideoSource(input_file_path, 0, index)
    depth_sources = depth_sources + [depth_source]
    ''' Number of frames '''
    frames_number = depth_source.properties.NumFrames
    frames_numbers = frames_numbers + [frames_number]
    ''' Width and height '''
    frame = depth_source.get_frame(0)
    frame_height = frame.EncodedHeight
    frame_width = frame.EncodedWidth
    frame_heights = frame_heights + [frame_height]
    frame_widths = frame_widths + [frame_width]

In [ ]:
''' Background 0 '''
background = np.ndarray(shape=(frame_heights[i], frame_widths[i]), dtype=np.float32)
source_idx = 0
backgrounds = get_background(depth_sources[source_idx],
                             frame_widths[source_idx],
                             frame_heights[source_idx],
                             np.linspace(0, frames_numbers[source_idx] - 1, num=100, dtype=np.int16))
plt.imshow(np.clip(backgrounds[source_idx],0.5, 0.65))
plt.colorbar()

In [ ]:
''' Assembly parameters '''
patch_width = 64
patch_height = 64
neighbour_offset = 1
number_of_left_neighbours = 2
number_of_right_neighbours = number_of_left_neighbours 

In [ ]:
start_datetime = datetime.now()
assemblies, indecies, skipped_indecies = get_assemblies(depth_source,
                                                       frame_width,
                                                       frame_height,
                                                       first_frame_idx,
                                                       last_frame_idx,
                                                       neighbour_offset,
                                                       number_of_left_neighbours,
                                                       number_of_right_neighbours,
                                                       patch_width,
                                                       patch_height,
                                                       background,
                                                       skipped_frames_limit,
                                                       previous_center,
                                                       max_center_movement)
end_datetime = datetime.now()

print(end_datetime - start_datetime)
print("Number of frames:\t" + str(frames_number))
print("Assemblies shape:\t" + str(assemblies.shape))
print("Skipped shape:\t" + str(skipped_indecies.shape))
print
print("Skipped")
print(skipped_indecies)
print

In [ ]:
assembly_offsets = np.arange(-number_of_left_neighbours*neighbour_offset, number_of_right_neighbours*neighbour_offset + 1, neighbour_offset)
print(assembly_offsets)
output_directory_path = "./processed_depth_data/" + "size_" + str(patch_width) + "_offset_" + str(neighbour_offset) + "_left_" + str(number_of_left_neighbours) + "_right_" + str(number_of_right_neighbours) + "/" 
output_file_path = output_directory_path + str(source_idx + 1) + ".pickle"
print(output_file_path)

In [ ]:
number_of_saved = 0
number_per_save = 4000
number_of_save = 1
while True:
    if number_of_saved >= assemblies.shape[0]:
        break
    from_idx = number_of_saved
    until_idx = from_idx + number_per_save
    if until_idx > assemblies.shape[0]:
        until_idx = assemblies.shape[0]
    print(from_idx, until_idx)
    write_processed_depth_data(output_directory_path + str(source_idx + 1) + "_" + str(number_of_save) + ".pickle",
                     frames_number,
                     assembly_offsets,
                     assemblies[from_idx:until_idx,:,:,:],
                     indecies[from_idx:until_idx])
    number_of_save += 1
    number_of_saved = until_idx